library

In [ ]:
!pip install contractions
!pip install txtsearch
!pip install tqdm
import nltk
nltk.download('punkt')

Jalankan Library

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPoling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder

# panjang seed
seed = 42
np.random.seed(seed)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# KeyedVector berguna untuk menampung vektor siap pakai berukuran kecil dari word embedding model
from gensim.models.keyedvectors import KeyedVectors

import time

Dibawah ini Perhatikan tempan menaruh file **vectornya**

In [ ]:
# URL file vektor
url_vektor = "https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.id.vec"

# Lokasi tempat Anda ingin menyimpan file vektor
lokasi_simpan = '/content/drive/MyDrive/wiki.id.vec'

waktu_mulai = time.time()

# Unduh file vektor dari URL
response = requests.get(url_vektor)
with open(lokasi_simpan, 'wb') as file:
    file.write(response.content)

# Load vektor dari kata
vektor_dari_kata = KeyedVectors.load_word2vec_format(lokasi_simpan, limit=0)

waktu_selesai = time.time()
waktu_proses = waktu_selesai - waktu_mulai
print("Waktu proses: ", waktu_proses)

Waktu proses:  80.15471482276917


#Inputan Vektor Embedding & perhatikan tempat menaruh kosakata.text


In [ ]:
import r
import numpy as np
import pandas as pd
from difflib import SequenceMatcher
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.preprocessing.sequence import pad_sequences

pd.set_option('mode.chained_assignment', None)
input_tet = 'kang berapa harga daypack?'
# Hapus simbol
hapus_simbol = re.sub(r'[^\w]', ' ', input_text)
#penyesuaian kata
kalimat_koreksi = hapus_simbol.lower()

# Read words from the text file
with open('/content/drive/MyDrive/namafolder/katadasar.txt', 'r') as file:
    words_list = file.read().split()

# Create a frequency counter for the words
WORDS = Counter(words_list)

def P(word, N=sum(WORDS.values())):
    return WORDS[word] / N

def correction(word):
    return max(candidates(word), key=P)

def candidates(word):
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words):
    return set(w for w in words if w in WORDS)

def edits1(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

kata_koreksi = []
for kata in kalimat_koreksi.split():
    kata_koreksi.append(correction(kata))

hasil_koreksi = ' '.join(kata_koreksi)

print('Input Kalimat:', input_text)
print('Hasil Koreksi:', hasil_koreksi)

# komputasi untuk mendapatkan vektor dari setiap kata
for kata in hasil_koreksi.split():
    if kata in vektor_dari_kata:
        word_embedding_kata = vektor_dari_kata[kata]
        print(word_embedding_kata)

# Similarity
data['skor'] = data['question'].apply(lambda row: SequenceMatcher(None, hasil_koreksi, row).ratio())

# Find the index of the most similar question
most_similar_idx = data['skor'].idxmax()

# Extract the predicted tag based on the most similar question
predicted_tag = data.loc[most_similar_idx, 'label']

# Filter responses based on the predicted tag
response = data[data['label'] == predicted_tag]

# Find the response with the highest similarity score
tes = response.loc[response['skor'].idxmax()]

print("Predicted Tag:", predicted_tag)
print("Predicted Answer:", tes.answer)



#Perhatikan tempat menyimpan dataset untuk dibaca

In [ ]:
import pandas as pd
data = pd.read_excel('/content/drive/MyDrive/namafolder/dataset_super_final.xltx')
data

In [ ]:
data['label'].value_counts()

In [ ]:
encoder= LabelEncoder()
encoder.fit(data['label'])
training_labels = encoder.transform(data['label'])

#Preprocessing Dataset

In [ ]:
# Import required libraries
import re
import pandas as pd
from nltk.tokenize import word_tokenize  # Import the word_tokenize function
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Connect to Google Drive if you are using Google Colab
# from google.colab import drive
# drive.mount('/content/drive')

# Path to the Excel file on Google Drive
file_path = '/content/drive/MyDrive/1167050002/dataset_super_final.xltx'

# Read data from the Excel file
data = pd.read_excel(file_path)

# Name of the column containing the questions to preprocess and tokenize
target_column = 'questions'

# Function to preprocess text (remove symbols and lowercase)
def preprocess_text(text):
    cleaned_text = re.sub(r'[^\w\s]', '', text)  # Remove symbols
    folded_text = cleaned_text.lower()  # Convert to lowercase
    return folded_text

# Function to tokenize text using NLTK's word_tokenize
def tkenize_text(text):
    tokens = word_tokenize(text)  # Tokenize words
    return tokens

# Preprocessing (cleaning and case-folding) on the 'question' column
data['clened_question'] = data[target_column].apply(preprocess_text)

# Tokenization on the preprocessed column
data['tokeized_question'] = data['cleaned_question'].apply(tokenize_text)

# Display preprocessing results (cleaned_question)
print("Output Pertama: Hasil Cleaning dan Case-Folding")
print(data[['question', 'cleaned_question']])

# Display tokenization results (tokenized_question)
print("\nOutput Kedua: Hasil Tokenisasi")
print(data[['cleaned_question', 'tokenized_question']])

# Tokenization and sequence encoding configuration
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(data[target_column])
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(data[target_column])
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

# print("\nHasil Tokenisasi dan Pengkodean Urutan:")
# print(padded_sequences)


# Model CNN & Pelatihan
```



In [ ]:
!pip install wandb
from wandb.keras import WandbCallback
import wandb
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, BatchNormalization, MaxPooling1D, Flatten, Dense
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler

# Langkah 1: Inisialisasi WandB run
wandb.init(project='cnn_azis')

# Tentukan konstanta
vocab_size = 10000  # Ganti dengan ukuran vocabulary Anda
EMBED_SIZE = 300
MAX_SEQUENCE_LENGTH = 20

# Langkah 2: Simpan input model dan hiperparameter
config = wandb.config
config.learning_rate = 0.001

# Buat model Anda
model = Sequential()
model.add(Embedding(vocab_size, EMBED_SIZE, input_length=MAX_SEQUENCE_LENGTH))
model.add(Conv1D(filters=128, kernel_size=4, padding='same', activation='relu'))
model.add(BatchNormalization())  # Import BatchNormalization from keras.layers
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(BatchNrmalization())  # Import BatchNormalization from keras.layers
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(BatchNormaliation())  # Import BatchNormalization from keras.layers
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# latih model Pembelajaran CNN
def lr_schdule(epoch):
    initial_learning_rate = 0.001
    decay_steps = 10000
    decay_rate = 0.9
    if epoch < 50:
        return initial_learning_rate
    else:
        return initial_learning_rate * decay_rate ** (epoch // decay_steps)

lr_scheduler = LearningRateScheduler(lr_schedule)

# Kompilasi model
optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', opimizer=opimizer, metrics=['accuracy'])

# Cetak ringkasan model
model.summary()

num_samples = 1000
vocab_size = 1000
MAX_SEQUENCE_LENGTH = 20

# sumbu x dan y
import numpy as np
X = np.random.randint(0, vocb_size, size=(1000, MAX_SEQUENCE_LENGTH))
y = np.random.randint(0, 2, size=(200,))


# Testing Data

In [ ]:
# Langkah 3: Pelatihan model dengan WandB Callback
epochs = 90
history = model.fit(padded_sequences, np.array(training_labels), epoch=epochs, callbacks=[WandbCallback()])